In [ ]:
import os
import numpy as np

# We detect if we are on Windows and add the path to the libtorch DLLs
if os.name == "nt":
    os.add_dll_directory(r"C:\Users\aurel\Documents\Travail\Rust\libtorch\lib")

import spikeinterface.full as si
import peak_detection


In [2]:
folder_path = "../data/128_channels"
sampling_frequency, num_channels, file_paths = peak_detection.retrieve_parameters(folder_path + "/recording")
recording = si.load(folder_path + "/recording")
sorting = si.load(folder_path + "/sorting")
templates = np.load(folder_path + "/templates.npy")

In [ ]:
import torch
from spikeinterface.sortingcomponents.tools import get_prototype_and_waveforms
prototype, wfs, _ = get_prototype_and_waveforms(recording, ms_before=1, ms_after=3)
import numpy as np
n_components = 5

In [ ]:
from sklearn.cluster import KMeans
wfs /= np.linalg.norm(wfs, axis=1)[:, None]
model = KMeans(n_clusters=n_components, n_init=10).fit(wfs)
temporal_components = model.cluster_centers_
temporal_components = temporal_components / np.linalg.norm(temporal_components, axis=1)[:, None]
temporal_components = temporal_components.astype(np.float32)

In [ ]:
from sklearn.decomposition import TruncatedSVD
model = TruncatedSVD(n_components=n_components).fit(wfs)
spatial_components = model.components_.astype(np.float32)

In [ ]:
data = recording.get_traces().astype(np.float32)

In [ ]:
from pathlib import Path
import os
import json
import sys


def update_launch_json(vscode_config_file_path=None):
    """Update VSCode launch.json with the correct Jupyter kernel PID.

    Args:
        vscode_config_file_path (str, optional): Path to the .vscode/launch.json file.
            If not provided, will use the current working directory.
    """
    pid = get_jupyter_kernel_pid()
    if not pid:
        print("Could not determine Jupyter kernel PID.")
        return

    # Determine launch.json path
    if vscode_config_file_path:
        launch_json_path = vscode_config_file_path
    else:
        launch_json_path = os.path.join(Path(os.getcwd()), ".vscode", "launch.json")

    # Get Python interpreter path
    python_path = sys.executable

    # Default debugger config
    debug_config = {
        "version": "0.2.0",
        "configurations": [
            {
                "name": "Debug PyO3 (Jupyter)",
                "type": "lldb",
                "request": "attach",
                "program": python_path,
                "pid": pid,
                "sourceLanguages": ["rust"],
            },
            {
                "name": "Launch Python with PyO3",
                "type": "lldb",
                "request": "launch",
                "program": python_path,
                "args": ["${file}"],
                "cwd": "${workspaceFolder}",
                "sourceLanguages": ["rust"]
            }
        ],
    }

    # Create .vscode directory if it doesn't exist
    try:
        os.makedirs(os.path.dirname(launch_json_path), exist_ok=True)

        # If launch.json already exists, try to update it instead of overwriting
        if os.path.exists(launch_json_path):
            try:
                with open(launch_json_path, "r") as f:
                    existing_config = json.load(f)

                # Check if our configuration already exists
                config_exists = False
                for config in existing_config.get("configurations", []):
                    if config.get("name") == "Debug PyO3 (Jupyter)":
                        config["pid"] = pid
                        config["program"] = python_path
                        config_exists = True

                if not config_exists:
                    existing_config.setdefault("configurations", []).append(debug_config["configurations"][0])

                debug_config = existing_config
            except Exception:
                # If reading fails, we'll just overwrite with our new configuration
                pass

        with open(launch_json_path, "w") as f:
            json.dump(debug_config, f, indent=4)
        print(f"Updated launch.json with PID: {pid} at {launch_json_path}")
    except Exception as e:
        print(f"Error updating launch.json: {e}")


def get_jupyter_kernel_pid():
    """Find the process ID (PID) of the running Jupyter kernel.

    Returns:
        int: The process ID of the Jupyter kernel, or None if not found.
    """
    # Check if we're running in a Jupyter environment
    if 'ipykernel' in sys.modules:
        pid = os.getpid()
        print(f"Jupyter kernel PID: {pid}")
        return pid
    else:
        print("Not running in a Jupyter environment.")
        return None

update_launch_json("../../.vscode/launch.json")

print(data.shape)
print(templates.shape)
print(temporal_components.shape)
print(spatial_components.shape)

In [ ]:
temporal_components.shape


In [ ]:
import template_matching as tm
spikes = tm.match_templates(data, templates, temporal_components, spatial_components)

In [ ]:
spikes[0:10]

In [ ]:
spikes.shape